In [9]:
!pip install transformers torch scikit-learn



<div style="direction: rtl; text-align: right; font-family: 'B Nazanin', 'Arial', sans-serif;">
    این دستور برای نصب کتابخانه‌های transformers، torch، و scikit-learn استفاده می‌شود. این کتابخانه‌ها برای کار با مدل‌های پردازش زبان طبیعی، شبکه‌های عصبی و ابزارهای یادگیری ماشین استفاده می‌شوند.
</div>


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_df = pd.read_csv('/content/dataTrain.csv')

test_df = pd.read_csv('/content/testTrain.csv')

train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)



<div style="direction: rtl; text-align: right; font-family: 'B Nazanin', 'Arial', sans-serif;">توضیح: ابتدا کتابخانه‌های pandas و train_test_split از scikit-learn بارگذاری می‌شوند. سپس داده‌ها از فایل‌های CSV خوانده می‌شوند. داده‌ها به دو مجموعه آموزشی و اعتبارسنجی تقسیم می‌شوند.
</div>


In [16]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def encode_data(text_list, max_length=512):
    return tokenizer.batch_encode_plus(
        text_list,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

train_encodings = encode_data(train_df['description'].tolist())
val_encodings = encode_data(val_df['description'].tolist())
test_encodings = encode_data(test_df['description'].tolist())



<div style="direction: rtl; text-align: right; font-family: 'B Nazanin', 'Arial', sans-serif;">توضیح: کتابخانه DistilBertTokenizer از transformers بارگذاری می‌شود. یک تابع encode_data برای توکنایز کردن متن‌ها ایجاد می‌شود. این تابع متن‌ها را به توکن‌ها تبدیل می‌کند و آن‌ها را به فرمت مناسب برای مدل BERT آماده می‌کند. سپس داده‌های آموزشی، اعتبارسنجی و تست توکنایز می‌شوند.
</div>


In [17]:
import torch
from torch.utils.data import Dataset, DataLoader

class MovieDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_labels = train_df['genre'].astype('category').cat.codes.tolist()
val_labels = val_df['genre'].astype('category').cat.codes.tolist()
test_labels = test_df['genre'].astype('category').cat.codes.tolist()

train_dataset = MovieDataset(train_encodings, train_labels)
val_dataset = MovieDataset(val_encodings, val_labels)
test_dataset = MovieDataset(test_encodings, test_labels)



<div style="direction: rtl; text-align: right; font-family: 'B Nazanin', 'Arial', sans-serif;">کتابخانه‌های torch و DataLoader بارگذاری می‌شوند. یک کلاس MovieDataset ایجاد می‌شود که داده‌های توکنایز شده و برچسب‌ها را مدیریت می‌کند. سپس برچسب‌ها به کدهای دسته‌بندی تبدیل شده و Datasetهای آموزشی، اعتبارسنجی و تست ایجاد می‌شوند.
</div>


In [18]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [19]:
from transformers import DistilBertForSequenceClassification

# Load DistilBERT model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=train_df['genre'].nunique())
model = model.to(device)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



<div style="direction: rtl; text-align: right; font-family: 'B Nazanin', 'Arial', sans-serif;">
توضیح: کتابخانه‌های DistilBertForSequenceClassification, AdamW, و get_linear_schedule_with_warmup از transformers بارگذاری می‌شوند. مدل DistilBERT برای دسته‌بندی توالی‌ها بارگذاری می‌شود. اگر GPU موجود باشد، مدل به دستگاه CUDA منتقل می‌شود. سپس DataLoader برای مجموعه‌های داده‌های آموزشی و اعتبارسنجی ایجاد می‌شود. بهینه‌ساز AdamW و تنظیم‌کننده یادگیری خطی تنظیم می‌شوند.
</div>


In [20]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=2e-5)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

total_steps = len(train_loader) * 3
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training function
def train_epoch(model, data_loader, optimizer, device, scheduler):
    model.train()
    total_loss = 0
    correct_predictions = 0

    progress_bar = tqdm(data_loader, desc="Training")

    for batch in progress_bar:
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(**inputs, labels=labels)  
        loss = outputs.loss
        logits = outputs.logits
        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix(loss=total_loss / len(progress_bar), accuracy=correct_predictions.double() / (len(progress_bar) * data_loader.batch_size))

    return correct_predictions.double() / len(data_loader.dataset), total_loss / len(data_loader)

# Evaluation function
def eval_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0

    progress_bar = tqdm(data_loader, desc="Evaluating")

    with torch.no_grad():
        for batch in progress_bar:
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(**inputs, labels=labels)  
            loss = outputs.loss
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            total_loss += loss.item()

            progress_bar.set_postfix(loss=total_loss / len(progress_bar), accuracy=correct_predictions.double() / (len(progress_bar) * data_loader.batch_size))

    return correct_predictions.double() / len(data_loader.dataset), total_loss / len(data_loader)

# Training loop
epochs = 3

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    train_acc, train_loss = train_epoch(model, train_loader, optimizer, device, scheduler)
    val_acc, val_loss = eval_model(model, val_loader, device)
    print(f'Train loss {train_loss}, accuracy {train_acc}')
    print(f'Validation loss {val_loss}, accuracy {val_acc}')


Epoch 1/3


Evaluating: 100%|██████████| 565/565 [01:14<00:00,  7.57it/s, accuracy=tensor(0.6597, device='cuda:0', dtype=torch.float64), loss=1.09]


Train loss 1.302177163436422, accuracy 0.6131072500861348
Validation loss 1.0941453255647051, accuracy 0.6604651162790698
Epoch 2/3


Evaluating: 100%|██████████| 565/565 [01:14<00:00,  7.58it/s, accuracy=tensor(0.6748, device='cuda:0', dtype=torch.float64), loss=1.07]


Train loss 0.8852045568058223, accuracy 0.729487621203918
Validation loss 1.065812353024968, accuracy 0.6755260243632337
Epoch 3/3


Evaluating: 100%|██████████| 565/565 [01:14<00:00,  7.59it/s, accuracy=tensor(0.6808, device='cuda:0', dtype=torch.float64), loss=1.1]


Train loss 0.6335652845413021, accuracy 0.8136535905891619
Validation loss 1.1027447583664836, accuracy 0.6815060908084164



<div style="direction: rtl; text-align: right; font-family: 'B Nazanin', 'Arial', sans-serif;">
توضیح: دو تابع train_epoch و eval_model برای آموزش و ارزیابی مدل ایجاد می‌شوند. این توابع داده‌ها را به مدل ورودی می‌دهند و دقت و خطای مدل را محاسبه می‌کنند.

حلقه آموزشی برای سه دوره تنظیم شده است. در هر دوره، مدل آموزش داده شده و بر روی داده‌های اعتبارسنجی ارزیابی می‌شود. دقت و خطای مدل برای هر دوره چاپ می‌شود.
</div>


In [21]:
torch.save(model, 'distilbert_model.pth')


In [32]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive/distilbert_model.pth'

torch.save(model, drive_path)


Mounted at /content/drive


In [33]:
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

def predict(model, data_loader, device):
    model.eval()
    preds = []

    progress_bar = tqdm(data_loader, desc="Predicting")

    with torch.no_grad():
        for batch in progress_bar:
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            outputs = model(**inputs)
            logits = outputs.logits
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())

    return preds

test_preds = predict(model, test_loader, device)


genre_mapping = dict(enumerate(test_df['genre'].astype('category').cat.categories))
predicted_genres = [genre_mapping[p] for p in test_preds]

test_df['predicted_genre'] = predicted_genres

print(test_df[['description', 'genre', 'predicted_genre']])



Predicting: 100%|██████████| 996/996 [02:01<00:00,  8.19it/s]

                                            description         genre  \
0      Sandro is a well-known journalist and he is c...        comedy   
1      A young boys life is changed when he's kidnap...         short   
2      On the coast of Yugoslavia lives fisherman Iv...         drama   
3      Crime TV show that is a mosaic of individual ...         crime   
4      Adam is a lost soul. He has lost his girlfrie...         short   
...                                                 ...           ...   
7963   Patrick O"Neal narrates this 'nostalgic' look...   documentary   
7964   Montmartre in the summertime. A group of stre...        comedy   
7965   "Bar Rescue" heads "Back to the Bar" to check...    reality-tv   
7966   The son of a Swedish man murdered in Germany ...   documentary   
7967   On air news reporter Robin Taylor several vid...        horror   

     predicted_genre  
0              drama  
1          adventure  
2              drama  
3              crime  
4       


<div style="direction: rtl; text-align: right; font-family: 'B Nazanin', 'Arial', sans-serif;">
توضیح: داده‌های تست به مدل ورودی داده می‌شوند و پیش‌بینی‌ها انجام می‌شود. سپس نتایج پیش‌بینی شده با برچسب‌های واقعی مقایسه می‌شوند.
</div>


In [35]:
import numpy as np
accuracy = np.sum(test_df['genre'] == test_df['predicted_genre']) / len(test_df)
print(accuracy)

0.6676706827309237
